## Pacotes Necessários

A seguir estão as bibliotecas necessárias para que os códigos a seguir rodem. Pode ser necessário instalar mais algumas. Nestes casos, recomenda-se copiar o erro ao tentar compilar e usar alguma IA para o suporte necessário.

In [ ]:
!pip install opencv-contrib-python

In [ ]:
!pip install opencv-contrib-python numpy

In [ ]:
!pip install pygame

In [ ]:
!pip install --upgrade pandas numpy

In [ ]:
!pip install python-docx

## 1. ArUco Para Impressão

Você pode encontrar marcadores ArUco no site: https://chev.me/arucogen/. É importante você ter alguns em mãos. A seguir apresentaremo um código que gera os ArUcos em um .docx. Caso não queira imprimir você pode acessar o endereço pelo seu celular e exibir o ArUco a partir dele para sua webcam.

### Tutorial de Uso


1) **O que o script faz**  
   Cria um documento **Word (.docx)** contendo uma grade de **marcadores ArUco** (IDs sequenciais), cada um com o **ID impresso abaixo**.  
   Útil para imprimir folhas de marcadores padronizadas.

---

2) **Arquivos gerados**  
- Um arquivo Word chamado **`folha_de_arucos.docx`** (você pode renomear alterando `NOME_ARQUIVO_DOCX`). Sairá no repositório do arquivo "Aruco_Python.ipynb".

---

3) **Parâmetros principais (você pode ajustar no topo do código)**

- `NUMERO_DE_MARCADORES = 24`  
  Quantos marcadores a folha terá (IDs irão de `0` até `NUMERO_DE_MARCADORES - 1`).
  
- `MARCADORES_POR_LINHA = 3`  
  Monta uma grade (n colunas). O número de linhas é calculado automaticamente.

- `TAMANHO_MARCADOR_CM = 5.0`  
  Tamanho **do lado** do marcador na página (em centímetros). O `python-docx` escala a imagem no Word para este tamanho.

- `PIXELS_MARCADOR = 300`  
  Resolução do PNG temporário usado para cada marcador. Quanto maior, mais nítido na impressão (o Word escala para o tamanho em cm).

- `dicionario = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)`  
  Dicionário de marcadores. Este suporta IDs de `0` a `249`.  
  (Se você gerar mais que 250, troque o dicionário por outro com mais IDs.)

- `NOME_ARQUIVO_DOCX = "folha_de_arucos.docx"`  
  Nome do arquivo final.

### Código Python

In [ ]:
import cv2
import cv2.aruco as aruco
import numpy as np
import os
import docx
from docx.shared import Cm # Para especificar o tamanho em centímetros
from docx.enum.text import WD_ALIGN_PARAGRAPH # Para centralizar o texto

# --- CONFIGURAÇÕES ---

# Quantidade total de marcadores a serem gerados
NUMERO_DE_MARCADORES = 12 # Normalmente cabem 12 por página 

# Layout no Documento
MARCADORES_POR_LINHA = 3
TAMANHO_MARCADOR_CM = 5.0 # Tamanho do lado do marcador em centímetros

# Dicionário ArUco a ser usado
dicionario = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)

# Qualidade do marcador em pixels (maior = melhor para impressão)
PIXELS_MARCADOR = 300

# Nome do arquivo de saída
NOME_ARQUIVO_DOCX = "folha_de_arucos.docx"

# ---------------------

def gerar_docx_arucos():
    """
    Gera um documento .docx com marcadores ArUco dispostos em uma tabela.
    """
    print("Iniciando a geração do documento .docx de marcadores ArUco...")

    document = docx.Document()
    num_linhas = (NUMERO_DE_MARCADORES + MARCADORES_POR_LINHA - 1) // MARCADORES_POR_LINHA
    tabela = document.add_table(rows=num_linhas, cols=MARCADORES_POR_LINHA)
    tabela.style = 'Table Grid'

    for id_marcador in range(NUMERO_DE_MARCADORES):
        
        # --- CORREÇÃO APLICADA AQUI ---
        # Usamos a nova função 'generateImageMarker' que cria e retorna a imagem do marcador.
        # Não precisamos mais da linha 'np.zeros' que criava uma imagem preta.
        img_marcador = aruco.generateImageMarker(dicionario, id_marcador, PIXELS_MARCADOR)
        
        # O resto do código continua igual...
        nome_arquivo_temp = f"temp_aruco_{id_marcador}.png"
        cv2.imwrite(nome_arquivo_temp, img_marcador)

        linha = id_marcador // MARCADORES_POR_LINHA
        coluna = id_marcador % MARCADORES_POR_LINHA
        
        celula = tabela.cell(linha, coluna)
        celula.text = ''
        
        run = celula.paragraphs[0].add_run()
        run.add_picture(nome_arquivo_temp, width=Cm(TAMANHO_MARCADOR_CM))
        celula.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        p = celula.add_paragraph(f'ID: {id_marcador}')
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        os.remove(nome_arquivo_temp)

    try:
        document.save(NOME_ARQUIVO_DOCX)
        print("-" * 30)
        print(f"SUCESSO! O arquivo '{NOME_ARQUIVO_DOCX}' foi gerado.")
        print(f"Contém {NUMERO_DE_MARCADORES} marcadores de tamanho {TAMANHO_MARCADOR_CM}x{TAMANHO_MARCADOR_CM} cm.")
        print("-" * 30)
    except Exception as e:
        print(f"Ocorreu um erro ao salvar o documento: {e}")

# Executa a função
gerar_docx_arucos()

## 2. Aruco + Áudio

### Tutorial de Uso


1. **O que o programa faz**  
   Detecta marcadores **ArUco** pela câmera e reproduz um arquivo de **áudio** associado ao **ID** do marcador, conforme definido em um arquivo de configuração (`.xlsx` ou `.csv`).

2. **Estrutura esperada**  
   - Arquivo de configuração: `config_audio.xlsx` **ou** `config_audio.csv`.  
   - Pasta com áudios: `audios/` (no mesmo diretório do notebook ou do script).  
   - Colunas obrigatórias no arquivo de configuração:  
     - **`Audio`** — nome do arquivo (ex.: `dragao.mp3`, `audio0.mp3`)  
     - **`ID_Aruco`** — ID inteiro do marcador (ex.: `0`, `1`, `2`)

3. **Selecionando a câmera**  
   Ajuste o índice `CAM_INDEX` no código (comece com `0`; se não funcionar, tente `1`, `2`, …).  
   Se não tiver webcam, use um app como **Iriun Webcam**, mantendo o celular na **mesma rede** do computador.  
   Exemplo de linha a ajustar:
       captura = cv2.VideoCapture(x)

4. **Executando com outro arquivo de configuração (opcional)**  
   Você pode informar o caminho via argumento:
       python seu_arquivo.py --config config_audio.csv
   Em Jupyter, flags internas são ignoradas com segurança porque o script usa `parse_known_args`.

5. **Como encerrar o programa**  
   - Na janela da webcam, pressione **q**.  
   - Ou, no Jupyter, use **Kernel ⇒ Restart & Clear Output**.

6. **Erros comuns e soluções**  
   - **“Arquivo de configuração não encontrado”**: verifique `ARQUIVO_CONFIG` ou passe `--config`.  
   - **Áudio não toca**: confirme se o arquivo existe em `audios/` e se o nome em **`Audio`** está correto.  
   - **Leitura de XLSX**: requer `openpyxl` instalado (`pip install openpyxl`).  
   - **CSV**: use cabeçalhos exatamente **`Audio`** e **`ID_Aruco`**.  
   - **Câmera não abre**: troque `CAM_INDEX` (0, 1, 2, …) ou feche outros apps que estejam usando a câmera.

---

**Dica:** dê nomes **descritivos** aos áudios (ex.: `dragao_rugido.mp3`) para facilitar a manutenção.  
Você pode gerar e imprimir marcadores ArUco em: https://chev.me/arucogen/

### Código Python

In [ ]:
import cv2
import cv2.aruco as aruco
import pygame
import os
import sys

# =========================
# CONFIGURAÇÕES PRINCIPAIS
# =========================

# Dicionário ArUco utilizado (4x4 com até 250 IDs)
DICIONARIO_ARUCO = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)

# Caminho do arquivo de configuração: .xlsx/.xlsm (via openpyxl) ou .csv (via csv)
ARQUIVO_CONFIG = "config_audio.xlsx"

# Pasta onde estão os arquivos de áudio (mp3/wav/ogg etc.)
PASTA_AUDIOS = "audios"

# Nome da janela de visualização
NOME_JANELA = "Detector ArUco com Áudio"

# Índice da câmera (0 = câmera padrão; ajuste conforme seu sistema)
CAM_INDEX = 2


def ler_config_sem_pandas(caminho: str) -> dict:
    """
    Lê o mapeamento ID_Aruco -> nome de arquivo de áudio sem depender de pandas.
    - Suporta .xlsx/.xlsm (via openpyxl) e .csv (via csv).
    - Exige colunas com cabeçalhos exatos: 'Audio' e 'ID_Aruco'.
    Retorna: dict {id_int: "nome_arquivo.mp3"}
    """
    if not os.path.exists(caminho):
        print(f"Erro: Arquivo de configuração '{caminho}' não encontrado.")
        return {}

    _, ext = os.path.splitext(caminho.lower())

    if ext in (".xlsx", ".xlsm"):
        try:
            import openpyxl  # leitura de planilhas Excel
        except ImportError:
            print("Erro: 'openpyxl' não está instalado. Instale com: pip install openpyxl")
            return {}

        try:
            wb = openpyxl.load_workbook(caminho, data_only=True)
            ws = wb.active

            # Cabeçalhos da primeira linha
            header_row = next(ws.iter_rows(min_row=1, max_row=1))
            headers = [str(c.value).strip() if c.value is not None else "" for c in header_row]

            # Índices das colunas obrigatórias
            try:
                idx_audio = headers.index("Audio")
                idx_id = headers.index("ID_Aruco")
            except ValueError:
                print(f"Erro: O arquivo '{caminho}' deve ter as colunas 'Audio' e 'ID_Aruco'.")
                return {}

            # Varre as linhas e monta o dicionário
            mapa = {}
            for row in ws.iter_rows(min_row=2):
                audio_cell = row[idx_audio].value
                id_cell = row[idx_id].value
                if audio_cell is None or id_cell is None:
                    continue
                audio = str(audio_cell).strip()
                try:
                    id_int = int(id_cell)
                except Exception:
                    continue
                if audio:
                    mapa[id_int] = audio

            if not mapa:
                print(f"Aviso: Nenhum mapeamento válido encontrado em '{caminho}'.")
            else:
                print(f"Configuração lida de '{caminho}' com {len(mapa)} entradas.")
            return mapa

        except Exception as e:
            print(f"Erro ao ler '{caminho}': {e}")
            return {}

    elif ext == ".csv":
        import csv
        try:
            with open(caminho, newline="", encoding="utf-8") as f:
                reader = csv.DictReader(f)

                # Verifica cabeçalhos
                if "Audio" not in reader.fieldnames or "ID_Aruco" not in reader.fieldnames:
                    print(f"Erro: O arquivo '{caminho}' deve ter as colunas 'Audio' e 'ID_Aruco'.")
                    return {}

                mapa = {}
                for row in reader:
                    audio = (row.get("Audio") or "").strip()
                    id_str = (row.get("ID_Aruco") or "").strip()
                    if not audio or not id_str:
                        continue
                    try:
                        id_int = int(id_str)
                    except Exception:
                        continue
                    mapa[id_int] = audio

                if not mapa:
                    print(f"Aviso: Nenhum mapeamento válido encontrado em '{caminho}'.")
                else:
                    print(f"Configuração lida de '{caminho}' com {len(mapa)} entradas.")
                return mapa

        except Exception as e:
            print(f"Erro ao ler CSV '{caminho}': {e}")
            return {}

    else:
        print("Erro: formato de arquivo não suportado. Use .xlsx, .xlsm ou .csv")
        return {}


def main():
    # 1) Lê a configuração ID_Aruco -> arquivo de áudio
    arquivos_audio_por_id = ler_config_sem_pandas(ARQUIVO_CONFIG)
    if not arquivos_audio_por_id:
        print("Nenhuma configuração válida encontrada. Encerrando o programa.")
        return

    # 2) Verifica a pasta de áudios e acusa ausentes
    if not os.path.isdir(PASTA_AUDIOS):
        print(f"Aviso: pasta '{PASTA_AUDIOS}' não existe. Crie-a ou ajuste PASTA_AUDIOS.")
    else:
        faltando = [fn for fn in arquivos_audio_por_id.values()
                    if not os.path.exists(os.path.join(PASTA_AUDIOS, fn))]
        if faltando:
            print("Atenção: os seguintes arquivos não foram encontrados na pasta de áudios:")
            for fn in faltando:
                print("  -", fn)

    # 3) Inicializa o mixer do pygame (responsável pelo áudio)
    try:
        pygame.mixer.init()
        print("Pygame Mixer inicializado.")
    except Exception as e:
        print(f"Aviso: falha ao inicializar o mixer do pygame: {e}")

    # 4) Abre a câmera
    captura = cv2.VideoCapture(CAM_INDEX)
    if not captura.isOpened():
        print(f"Erro: Nao foi possivel abrir a camera no índice {CAM_INDEX}.")
        return

    # 5) Configura detector ArUco
    parametros_aruco = aruco.DetectorParameters()
    detector_aruco = aruco.ArucoDetector(DICIONARIO_ARUCO, parametros_aruco)

    # Controla qual ID tocou por último (para evitar repetição a cada frame)
    ultimo_id_tocado = -1

    # 6) Loop de captura e detecção
    while True:
        sucesso, img = captura.read()
        if not sucesso:
            print("Erro ao ler o frame da camera.")
            break

        # Converte para tons de cinza e detecta marcadores
        img_cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cantos_marcadores, ids_marcadores, _ = detector_aruco.detectMarkers(img_cinza)

        marcador_visivel_agora = -1

        if ids_marcadores is not None:
            # Desenha os marcadores detectados para visualização
            aruco.drawDetectedMarkers(img, cantos_marcadores, ids_marcadores)

            # Percorre IDs detectados; toca apenas quando um novo ID aparece
            for id_marcador in ids_marcadores.flatten():
                if marcador_visivel_agora == -1:
                    marcador_visivel_agora = id_marcador

                if id_marcador in arquivos_audio_por_id and id_marcador != ultimo_id_tocado:
                    nome_arquivo = arquivos_audio_por_id[id_marcador]
                    caminho_completo_audio = os.path.join(PASTA_AUDIOS, nome_arquivo)
                    print(f"Marcador ID {id_marcador} detectado. Tocando '{caminho_completo_audio}'...")

                    try:
                        pygame.mixer.music.load(caminho_completo_audio)
                        pygame.mixer.music.play()
                        ultimo_id_tocado = id_marcador
                    except Exception as e:
                        print(f"Falha ao tocar '{caminho_completo_audio}': {e}")

        # Se nenhum marcador ficar visível no frame atual, libera para tocar novamente
        if marcador_visivel_agora == -1:
            ultimo_id_tocado = -1

        # Mostra a imagem na janela
        cv2.imshow(NOME_JANELA, img)

        # Saída pelo teclado
        tecla = cv2.waitKey(1) & 0xFF
        if tecla == ord('q'):
            print("Tecla 'q' pressionada. Encerrando...")
            break

        # Saída ao fechar a janela
        if cv2.getWindowProperty(NOME_JANELA, cv2.WND_PROP_VISIBLE) < 1:
            print("Janela fechada pelo usuário. Encerrando...")
            break

    # 7) Limpeza de recursos
    print("Finalizando e liberando recursos...")
    captura.release()
    cv2.destroyAllWindows()
    pygame.quit()
    print("Programa finalizado.")


if __name__ == "__main__":
    # Aceita --config/-c para informar o caminho do arquivo de configuração.
    # parse_known_args é usado para ignorar flags injetadas pelo Jupyter (ex.: '-f ...').
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "-c", "--config",
        default=ARQUIVO_CONFIG,
        help="Caminho para o arquivo de configuração (xlsx/xlsm/csv) com colunas 'Audio' e 'ID_Aruco'."
    )
    args, _ = parser.parse_known_args()
    ARQUIVO_CONFIG = args.config
    main()

## 3. Aruco + Imagem/Gif

### Tutorial de Uso

1. **O que o programa faz**  
   Detecta marcadores **ArUco** pela câmera e sobrepõe **imagens** ou **GIFs** nos marcadores correspondentes.  
   A associação **ID → arquivo** e o **fator de escala** de cada asset vêm de um arquivo de configuração (`.xlsx`/`.xlsm` ou `.csv`).

2. **Estrutura esperada**  
   - Arquivo de configuração: `config_img.xlsx` **ou** `config_img.csv`  
   - Pasta de mídias: `midias/` (ao lado do arquivo de configuração)  
   - Colunas obrigatórias na planilha:  
     - **`Arquivo`** — caminho do asset (ex.: `dragao.png`, `fogo.gif`)  
     - **`ID_Aruco`** — ID inteiro do marcador (ex.: `0`, `1`, `2`)  
     - **`Escala`** *(opcional)* — quanto ampliar/reduzir o overlay:  
       - `1.0` → ocupa exatamente a área do ArUco  
       - `2.0` → **duas vezes maior**  
       - `0.5` → **metade**  
       > Se a coluna/célula estiver ausente/vazia, usa `1.0`.  
   - Os arquivos podem estar em `midias/` ou em qualquer pasta; use caminho **relativo** ou **absoluto** em **`Arquivo`**.

3. **Selecionando a câmera**  
   Ajuste **`CAM_INDEX`** no código (comece com `0`; se não funcionar, tente `1`, `2`, …).  
   Se não tiver webcam, pode usar um app como **Iriun Webcam**, mantendo o celular na **mesma rede**.

   Exemplo de linha a ajustar:

       captura = cv2.VideoCapture(CAM_INDEX)

4. **Como a escala é aplicada**  
   - O código lê **`Escala`** da planilha por **asset** e calcula os cantos **escalonados** em torno do centro do marcador.  
   - O overlay (imagem/GIF) é então **transformado em perspectiva** para encaixar nesse quadrilátero escalado.  
   - Resultado: você controla o “tamanho” do overlay **por item** apenas editando a planilha.

5. **GIFs animados e imagens com transparência**  
   - **GIF**: todos os frames são carregados e exibidos em sequência enquanto o marcador estiver visível.  
   - **PNG com alfa**: a transparência é respeitada.  
   - Antes de desenhar, a região destino recebe a cor **`COR_FUNDO`** (padrão: branca).

6. **Estabilização (menos tremor)**  
   Esta versão inclui três recursos para suavizar o overlay:  
   - **EMA dos cantos** (`ALPHA_SUAVIZACAO`): média móvel exponencial nos vértices detectados para reduzir ruído quadro a quadro.  
   - **Hold** (`HOLD_FRAMES`): mantém o último overlay por alguns frames após perder o marcador, evitando “piscadas”.  
   - **Refino sub-pixel** (`REFINAR_SUBPIX`): melhora a precisão dos cantos quando suportado pelo OpenCV.  
   > Dica: escalas muito altas amplificam ruído. Se notar tremor, use valores moderados (ex.: `1.5–3`) e aumente levemente `ALPHA_SUAVIZACAO` e/ou `HOLD_FRAMES`.

7. **Executando**  
   - Garanta que `config_img.xlsx` (ou `.csv`) e a pasta `midias/` estejam no projeto.  
   - Preencha a planilha conforme o item 2.  
   - Rode o notebook/script: a janela da câmera abre e os overlays aparecem ao detectar os IDs.

8. **Como encerrar o programa**  
   - Na janela da webcam, pressione **`q`**.  
   - Ou, no Jupyter, use **Kernel ⇒ Restart & Clear Output**.

9. **Erros comuns e soluções**  
   - **“Arquivo de configuração não encontrado”**: verifique **`ARQUIVO_CONFIG`** e o caminho do arquivo.  
   - **Colunas ausentes**: confirme os cabeçalhos **`Arquivo`**, **`ID_Aruco`** e (opcional) **`Escala`**.  
   - **Imagem/GIF não carrega**: confira o caminho em **`Arquivo`** e a extensão (suporta PNG/JPG/GIF).  
   - **Câmera não abre**: troque **`CAM_INDEX`** (0, 1, 2, …) ou feche outros apps que estejam usando a câmera.  
   - **Overlay tremendo**: aumente `ALPHA_SUAVIZACAO`, eleve `HOLD_FRAMES`, melhore iluminação/foco e mantenha o marcador plano.

---

**Dica:** para manter tudo organizado, use nomes de arquivos descritivos (ex.: `dragao_frente.png`, `fogo_loop.gif`).  
Você pode gerar e imprimir marcadores ArUco em: https://chev.me/arucogen/

### Código Python

In [ ]:
import cv2
import cv2.aruco as aruco
import numpy as np
from PIL import Image
import pandas as pd
import os
from typing import List, Tuple, Optional, Dict, Any

# ============================================
# CONFIGURAÇÃO PRINCIPAL
# ============================================

DICIONARIO_ARUCO = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
ARQUIVO_CONFIG = "config_img.xlsx"      # planilha com: Arquivo, ID_Aruco, Escala
PASTA_MIDIAS   = "midias"               # pasta (relativa à planilha) com PNG/JPG/GIF
NOME_JANELA    = "Aruco -> Imagem"
COR_FUNDO      = (255, 255, 255)        # BGR
CAM_INDEX      = 2                      # 0,1,2...

# ============================================
# AJUSTES DE ESTABILIDADE
# ============================================

ALPHA_SUAVIZACAO = 0.8   # 0..1 (quanto maior, mais liso e mais “inércia”)
HOLD_FRAMES      = 10      # quantos frames manter após perder o marcador
REFINAR_SUBPIX   = True   # refino sub-pixel dos cantos (ajuda bastante)

# ============================================
# Caminhos
# ============================================

def obter_caminho_midia(nome_arquivo: str) -> str:
    if os.path.isabs(nome_arquivo):
        return nome_arquivo
    pasta_cfg = os.path.dirname(os.path.abspath(ARQUIVO_CONFIG))
    candidatos = [
        os.path.join(pasta_cfg, PASTA_MIDIAS, nome_arquivo),
        os.path.join(pasta_cfg, nome_arquivo),
        os.path.join(os.getcwd(), PASTA_MIDIAS, nome_arquivo),
        os.path.join(os.getcwd(), nome_arquivo),
    ]
    for c in candidatos:
        if os.path.exists(c):
            return c
    return candidatos[0]

# ============================================
# Leitura da planilha (.xlsx/.xlsm/.csv)
# ============================================

def _ler_csv(nome_arquivo: str) -> List[Tuple[str, int, float]]:
    df = pd.read_csv(nome_arquivo)
    return _validar_df(df, nome_arquivo)

def _ler_xlsx_via_pandas(nome_arquivo: str) -> List[Tuple[str, int, float]]:
    df = pd.read_excel(nome_arquivo)
    return _validar_df(df, nome_arquivo)

def _ler_xlsx_via_openpyxl(nome_arquivo: str) -> List[Tuple[str, int, float]]:
    try:
        import openpyxl
    except ImportError:
        print("Erro: openpyxl não está instalado. Instale com: pip install openpyxl")
        return []
    wb = openpyxl.load_workbook(nome_arquivo, data_only=True)
    ws = wb.active
    headers = [str(c.value).strip() if c.value is not None else "" for c in next(ws.iter_rows(min_row=1, max_row=1))]
    try:
        idx_arquivo = headers.index("Arquivo")
        idx_id = headers.index("ID_Aruco")
    except ValueError:
        print(f"Erro: O arquivo '{nome_arquivo}' deve ter as colunas 'Arquivo' e 'ID_Aruco'.")
        return []
    idx_escala = headers.index("Escala") if "Escala" in headers else None
    config_list: List[Tuple[str, int, float]] = []
    for row in ws.iter_rows(min_row=2):
        arquivo = row[idx_arquivo].value if row[idx_arquivo] is not None else ""
        id_val  = row[idx_id].value
        escala  = 1.0
        if idx_escala is not None and row[idx_escala] is not None and row[idx_escala].value is not None:
            try: escala = float(row[idx_escala].value)
            except: escala = 1.0
        if arquivo == "" or id_val is None:
            continue
        try:
            config_list.append((str(arquivo), int(id_val), float(escala)))
        except: continue
    print(f"Configuração (fallback openpyxl) lida de '{nome_arquivo}' com sucesso.")
    return config_list

def _validar_df(df: pd.DataFrame, nome_arquivo: str) -> List[Tuple[str, int, float]]:
    if "Arquivo" not in df.columns or "ID_Aruco" not in df.columns:
        print(f"Erro: O arquivo '{nome_arquivo}' deve ter as colunas 'Arquivo' e 'ID_Aruco'.")
        return []
    config_list: List[Tuple[str, int, float]] = []
    for _, row in df.iterrows():
        escala = row.get("Escala", 1.0)
        if pd.isna(escala): escala = 1.0
        try:
            config_list.append((str(row["Arquivo"]), int(row["ID_Aruco"]), float(escala)))
        except: pass
    print(f"Configuração lida de '{nome_arquivo}' com sucesso.")
    return config_list

def ler_config_planilha(nome_arquivo: str) -> List[Tuple[str, int, float]]:
    if not os.path.exists(nome_arquivo):
        print(f"Erro: Arquivo de configuração '{nome_arquivo}' não encontrado.")
        return []
    ext = os.path.splitext(nome_arquivo)[1].lower()
    try:
        if ext == ".csv":
            return _ler_csv(nome_arquivo)
        elif ext in (".xlsx", ".xlsm"):
            try:
                return _ler_xlsx_via_pandas(nome_arquivo)
            except Exception as e:
                msg = str(e).lower()
                if "openpyxl" in msg and ("requires version" in msg or "installed" in msg):
                    print("Aviso: Problema de versão do openpyxl detectado no pandas. Usando fallback via openpyxl puro.")
                    return _ler_xlsx_via_openpyxl(nome_arquivo)
                else:
                    print(f"Ocorreu um erro ao ler a planilha com pandas: {e}")
                    print("Tentando fallback via openpyxl puro...")
                    return _ler_xlsx_via_openpyxl(nome_arquivo)
        else:
            print("Erro: formato de arquivo não suportado. Use .xlsx/.xlsm ou .csv")
            return []
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo de configuração: {e}")
        return []

# ============================================
# Carregamento de mídia (GIF/PNG/JPG) — BGRA
# ============================================

def carregar_frames_gif(nome_arquivo: str) -> Optional[List[np.ndarray]]:
    try:
        caminho = obter_caminho_midia(nome_arquivo)
        gif = Image.open(caminho)
        frames: List[np.ndarray] = []
        for i in range(getattr(gif, "n_frames", 1)):
            gif.seek(i)
            rgba = gif.convert("RGBA")
            frame = cv2.cvtColor(np.array(rgba), cv2.COLOR_RGBA2BGRA)
            frames.append(frame)
        print(f"-> GIF '{nome_arquivo}' carregado com {len(frames)} frames. ({caminho})")
        return frames
    except Exception as e:
        print(f"Erro ao carregar o GIF '{nome_arquivo}': {e}")
        return None

def carregar_imagem_estatica(nome_arquivo: str) -> Optional[np.ndarray]:
    try:
        caminho = obter_caminho_midia(nome_arquivo)
        img = cv2.imread(caminho, cv2.IMREAD_UNCHANGED)
        if img is None:
            raise FileNotFoundError(f"Não foi possível abrir '{caminho}'")
        if len(img.shape) == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        if img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
        print(f"-> Imagem '{nome_arquivo}' carregada. ({caminho})")
        return img
    except Exception as e:
        print(f"Erro ao carregar a imagem '{nome_arquivo}': {e}")
        return None

# ============================================
# Geometria e composição
# ============================================

def calcular_cantos_escalonados(cantos_marcador: np.ndarray, fator_escala: float) -> np.ndarray:
    if fator_escala == 1.0:
        return cantos_marcador.astype(np.float32)
    centro = cantos_marcador.mean(axis=0)
    cantos_escalonados = []
    for canto in cantos_marcador:
        vetor = canto - centro
        novo_canto = centro + vetor * fator_escala
        cantos_escalonados.append(novo_canto)
    return np.array(cantos_escalonados, dtype=np.float32)

def sobrepor_asset(img_fundo: np.ndarray,
                   asset_sobreposicao: np.ndarray,
                   cantos_destino: np.ndarray,
                   cor_fundo_desejada: Tuple[int, int, int]) -> np.ndarray:
    cv2.fillConvexPoly(img_fundo, cantos_destino.astype(np.int32), cor_fundo_desejada)
    h, w = asset_sobreposicao.shape[:2]
    pts_origem = np.array([[0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1]], dtype=np.float32)
    H, _ = cv2.findHomography(pts_origem, cantos_destino.astype(np.float32))
    img_warp = cv2.warpPerspective(asset_sobreposicao, H, (img_fundo.shape[1], img_fundo.shape[0]),
                                   flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=COR_FUNDO)
    if img_warp.shape[2] == 3:
        alpha = np.ones((img_warp.shape[0], img_warp.shape[1]), dtype=np.float32)
    else:
        alpha = img_warp[:, :, 3].astype(np.float32) / 255.0
    mascara = np.dstack([alpha]*3)
    return ((1 - mascara) * img_fundo.astype(np.float32) + mascara * img_warp[:, :, :3].astype(np.float32)).astype(np.uint8)

# ============================================
# Estabilização temporal (EMA + hold)
# ============================================

def suavizar_cantos(prev: Optional[np.ndarray], atual: np.ndarray, alpha: float) -> np.ndarray:
    """Exponential Moving Average nos cantos (4x2)."""
    if prev is None:
        return atual.astype(np.float32)
    return (alpha * prev.astype(np.float32) + (1.0 - alpha) * atual.astype(np.float32)).astype(np.float32)

# ============================================
# Execução
# ============================================

def main():
    # 1) Lê configuração
    cfg = ler_config_planilha(ARQUIVO_CONFIG)
    if not cfg:
        print("Nenhuma configuração válida encontrada. Encerrando o programa.")
        return

    # 2) Pré-carrega assets
    print("\nIniciando o carregamento dos assets...")
    assets: Dict[int, Dict[str, Any]] = {}
    gif_idx: Dict[int, int] = {}
    for arquivo, ar_id, escala in cfg:
        data = None
        if str(arquivo).lower().endswith(".gif"):
            data = carregar_frames_gif(arquivo)
            if data: gif_idx[ar_id] = 0
        else:
            data = carregar_imagem_estatica(arquivo)
        if data is not None:
            assets[ar_id] = {"asset": data, "escala": float(escala)}
        else:
            print(f"Aviso: asset '{arquivo}' (ID {ar_id}) não foi carregado.")
    if not assets:
        print("Nenhum asset foi carregado com sucesso.")
        return
    print("Carregamento concluído.\n")

    # 3) Câmera e detector
    cap = cv2.VideoCapture(CAM_INDEX)
    if not cap.isOpened():
        print(f"Erro: Não foi possível abrir a câmera no índice {CAM_INDEX}.")
        return

    params = aruco.DetectorParameters()
    if REFINAR_SUBPIX:
        # ativa refino sub-pixel (requer imagem cinza)
        try:
            params.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
        except Exception:
            pass
    detector = aruco.ArucoDetector(DICIONARIO_ARUCO, params)

    # Estado para suavização/hold
    suavizados: Dict[int, np.ndarray] = {}     # cantos suavizados por ID
    last_seen: Dict[int, int] = {}             # frame em que cada ID foi visto pela última vez
    frame_count = 0

    while True:
        ok, frame = cap.read()
        if not ok:
            print("Aviso: falha ao ler frame da câmera.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cantos, ids, _ = detector.detectMarkers(gray)
        vistos_este_frame = set()

        # 4) Atualiza suavização para IDs detectados
        if ids is not None:
            for i, ar_id in enumerate(ids.flatten()):
                if ar_id in assets:
                    corners = cantos[i][0]  # (4,2)
                    # refino sub-pixel manual (extra), se disponível
                    if REFINAR_SUBPIX and corners.dtype != np.float32:
                        corners = corners.astype(np.float32)

                    suavizados[ar_id] = suavizar_cantos(suavizados.get(ar_id), corners, ALPHA_SUAVIZACAO)
                    last_seen[ar_id] = frame_count
                    vistos_este_frame.add(ar_id)

                    # avança GIF somente quando o marcador está visível
                    if isinstance(assets[ar_id]["asset"], list):
                        idx = gif_idx[ar_id]
                        gif_idx[ar_id] = (idx + 1) % len(assets[ar_id]["asset"])

        # 5) Renderiza: IDs visíveis OU ainda dentro da janela de hold
        for ar_id, dados in assets.items():
            ainda_no_hold = (ar_id in last_seen) and (frame_count - last_seen[ar_id] <= HOLD_FRAMES)
            if (ar_id in vistos_este_frame) or ainda_no_hold:
                asset = dados["asset"]
                escala = dados["escala"]

                # escolhe frame do GIF (se for o caso)
                if isinstance(asset, list):
                    idx = gif_idx[ar_id]
                    frame_asset = asset[idx if ar_id in vistos_este_frame else (idx % len(asset))]
                else:
                    frame_asset = asset

                if ar_id in suavizados:
                    cantos_suaves = suavizados[ar_id]
                    cantos_dest = calcular_cantos_escalonados(cantos_suaves, escala)
                    frame = sobrepor_asset(frame, frame_asset, cantos_dest, COR_FUNDO)

        cv2.imshow(NOME_JANELA, frame)
        frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Programa finalizado.")

if __name__ == "__main__":
    main()
